In [1]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [2]:
def collect_stock_data():
    """
    Collects stock data based on user inputs
    """
    print("=" * 60)
    print("STOCK TRADING STRATEGY ANALYZER")
    print("=" * 60)
    
    # Get user inputs
    ticker_input = input("Please enter the stock ticker you have chosen to invest in: \n")
    
    print("\n" + "-" * 60)
    start_date = input("Enter the start date (format: YYYY-MM-DD):\n")
    
    print("\n" + "-" * 60)
    end_date = input("Enter the end date (format: YYYY-MM-DD):\n")
    
    print("\n" + "-" * 60)
    print("MOVING AVERAGE PERIODS")
    print("-" * 60)
    ma_list = []
    
    ma_value = int(input("What would you like the 1st period to be for your moving average?\n"))
    ma_list.append(ma_value)
    
    ma_value = int(input("What would you like the 2nd period to be for your moving average?\n"))
    ma_list.append(ma_value)
    
    # Download stock data
    print(f"\nDownloading data for {ticker_input}...")
    try:
        ticker_data_full = yf.download(ticker_input, start=start_date, end=end_date)
        
        if ticker_data_full.empty:
            print("No data found. Please check your ticker symbol and date range.")
            return None, None, None
        
        # Use Adjusted Close price
        ticker_data = ticker_data_full['Adj Close'].copy()
        
        print(f"Successfully downloaded {len(ticker_data)} days of data")
        print(f"Date range: {ticker_data.index[0].date()} to {ticker_data.index[-1].date()}")
        
        return ticker_input, ticker_data, ma_list
        
    except Exception as e:
        print(f"Error downloading data: {e}")
        return None, None, None

In [3]:
def simple_moving_average(ticker_data, length, start):
    """
    Calculate simple moving average for a given period
    """
    s = 0
    for i in range(0, int(length)):
        s += ticker_data[start - i]
    return s / length

def calculate_moving_averages(ticker_input, ticker_data, ma_list):
    """
    Calculate moving averages for specified periods
    """
    moving_averages = pd.DataFrame(index=ticker_data.index)
    
    for i in ma_list:
        list2 = []
        for v in range(0, len(ticker_data.index)):
            if v < i - 1:
                list2.append(0)
            if v > i - 1 or v == i - 1:
                list2.append(simple_moving_average(ticker_data, i, v))
        
        moving_averages[ticker_input + " " + str(i) + " Day Moving Average"] = list2
    
    # Remove all values that are NA and make sure all lists start from the same date
    ticker_data_clean = ticker_data.iloc[max(ma_list)-1:].copy()
    moving_averages_clean = moving_averages.iloc[max(ma_list)-1:].copy()
    
    return ticker_data_clean, moving_averages_clean

In [4]:
def generate_trading_signals(moving_averages, ticker_data_clean):
    """
    Generate buy/sell signals based on moving average crossovers
    """
    # Calculate the difference between moving averages
    def difference_calc(x):
        return x[1] - x[0]
    
    differences = pd.DataFrame(index=ticker_data_clean.index)
    differences = moving_averages.apply(difference_calc, axis=1)
    
    # Test for sign differences to sense crossovers
    def test(x):
        if differences.iloc[x] < 0 and differences.iloc[x-1] > 0:
            return "sell"
        if differences.iloc[x] > 0 and differences.iloc[x-1] < 0:
            return "buy"
        if differences.iloc[x] > 0 and differences.iloc[x-1] > 0:
            return "continue_s"
        if differences.iloc[x] < 0 and differences.iloc[x-1] < 0:
            return "continue_b"
        return "hold"
    
    # Generate signals
    signals = []
    positions = []
    current_position = None
    
    # Initialize position based on first difference
    if differences.iloc[0] > 0:
        current_position = "bought"
    elif differences.iloc[0] < 0:
        current_position = "sold"
    else:
        current_position = "neutral"
    
    for r in range(0, len(ticker_data_clean.index)):
        signal = test(r)
        signals.append(signal)
        
        # Update position
        if signal == "buy":
            current_position = "bought"
        elif signal == "sell":
            current_position = "sold"
        
        positions.append(current_position)
    
    return differences, signals, positions

In [5]:
def calculate_performance(ticker_input, ticker_data_clean, signals, positions):
    """
    Calculate performance of trading strategy vs buy-and-hold
    """
    # Calculate continuous investing (buy-and-hold)
    initial_price = ticker_data_clean.iloc[0]
    continuous_investing = []
    
    for i in range(len(ticker_data_clean)):
        if i == 0:
            continuous_investing.append(0)
        else:
            # Normalized return from initial investment
            return_pct = (ticker_data_clean.iloc[i] - initial_price) / initial_price * 100
            continuous_investing.append(return_pct)
    
    # Calculate indicative investing (strategy-based)
    indicative_investing = []
    last_true = 0
    
    # Initialize position
    position = "bought" if positions[0] == "bought" else "sold"
    
    for r in range(len(ticker_data_clean)):
        if r == 0:
            indicative_investing.append(0)
            continue
            
        signal = signals[r]
        
        if position == "bought" and (signal == "continue_b" or signal == "continue_s"):
            # Calculate return while holding
            daily_return = (ticker_data_clean.iloc[r] - ticker_data_clean.iloc[r-1]) / ticker_data_clean.iloc[r-1] * 100
            last_true = last_true + daily_return
            indicative_investing.append(last_true)
            
        elif position == "bought" and signal == "sell":
            # Exit position
            daily_return = (ticker_data_clean.iloc[r] - ticker_data_clean.iloc[r-1]) / ticker_data_clean.iloc[r-1] * 100
            last_true = last_true + daily_return
            indicative_investing.append(last_true)
            position = "sold"
            
        elif position == "sold" and signal == "buy":
            # Enter position
            indicative_investing.append(last_true)
            position = "bought"
            
        else:
            indicative_investing.append(last_true)
    
    # Create comparison dataframe
    comparison = pd.DataFrame(index=ticker_data_clean.index)
    comparison[ticker_input + ' Continuous Investing (%)'] = continuous_investing
    comparison[ticker_input + ' Indicative Investing (%)'] = indicative_investing
    
    return comparison

In [6]:
def standard_deviation(data_list):
    """
    Calculate standard deviation of a data list
    """
    if len(data_list) <= 1:
        return 0
    
    import statistics
    new_list = data_list
    standard_deviation_sum = 0
    
    for i in range(0, len(new_list)):
        standard_deviation_sum += (new_list[i] - statistics.mean(new_list)) ** 2
    
    return ((standard_deviation_sum / (len(new_list) - 1)) ** (0.5))

def outperformance_percentage(comparison, ticker_input):
    """
    Calculate percentage of days strategy outperforms buy-and-hold
    """
    outperformance_count = 0
    total_count = 0
    
    for r in range(0, len(comparison.index)):
        if comparison[ticker_input + ' Indicative Investing (%)'].iloc[r] > comparison[ticker_input + ' Continuous Investing (%)'].iloc[r]:
            outperformance_count += 1
        total_count += 1
    
    outperformance_percentage = (outperformance_count / total_count) * 100
    return outperformance_percentage

def calculate_statistics(comparison, ticker_input):
    """
    Calculate comprehensive statistics for both strategies
    """
    stats = {}
    
    # Strategy returns
    strat_returns = comparison[ticker_input + ' Indicative Investing (%)']
    # Buy-and-hold returns
    bh_returns = comparison[ticker_input + ' Continuous Investing (%)']
    
    # Final returns
    stats['Strategy Final Return (%)'] = strat_returns.iloc[-1]
    stats['Buy-and-Hold Final Return (%)'] = bh_returns.iloc[-1]
    
    # Standard deviations (risk)
    stats['Strategy Std Dev'] = standard_deviation(strat_returns.tolist())
    stats['Buy-and-Hold Std Dev'] = standard_deviation(bh_returns.tolist())
    
    # Risk-adjusted returns (Sharpe-like, assuming 0 risk-free rate)
    if stats['Strategy Std Dev'] > 0:
        stats['Strategy Risk-Adjusted Return'] = stats['Strategy Final Return (%)'] / stats['Strategy Std Dev']
    else:
        stats['Strategy Risk-Adjusted Return'] = 0
    
    if stats['Buy-and-Hold Std Dev'] > 0:
        stats['Buy-and-Hold Risk-Adjusted Return'] = stats['Buy-and-Hold Final Return (%)'] / stats['Buy-and-Hold Std Dev']
    else:
        stats['Buy-and-Hold Risk-Adjusted Return'] = 0
    
    # Outperformance percentage
    stats['Outperformance Percentage (%)'] = outperformance_percentage(comparison, ticker_input)
    
    return pd.DataFrame.from_dict(stats, orient='index', columns=['Value'])

In [7]:
def plot_price_and_moving_averages(ticker_input, ticker_data_clean, moving_averages_clean):
    """
    Plot stock price with moving averages
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), height_ratios=[2, 1])
    
    # Plot 1: Price and Moving Averages
    ax1.plot(ticker_data_clean.index, ticker_data_clean.values, 
             label=f'{ticker_input} Price', linewidth=2, color='black', alpha=0.7)
    
    colors = ['blue', 'red']
    for i, col in enumerate(moving_averages_clean.columns):
        ax1.plot(moving_averages_clean.index, moving_averages_clean[col].values, 
                 label=col, linewidth=1.5, alpha=0.8, color=colors[i % len(colors)])
    
    ax1.set_title(f'{ticker_input} - Price and Moving Averages', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Price ($)', fontsize=12)
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Moving Average Difference
    differences = moving_averages_clean.iloc[:, 1] - moving_averages_clean.iloc[:, 0]
    ax2.fill_between(differences.index, differences.values, 0, 
                     where=differences.values >= 0, color='green', alpha=0.3, label='MA1 > MA2')
    ax2.fill_between(differences.index, differences.values, 0, 
                     where=differences.values < 0, color='red', alpha=0.3, label='MA1 < MA2')
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax2.set_title('Moving Average Difference', fontsize=14)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.set_ylabel('Difference', fontsize=12)
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_trading_signals(ticker_data_clean, signals, positions):
    """
    Plot trading signals on price chart
    """
    plt.figure(figsize=(14, 7))
    
    # Plot price
    plt.plot(ticker_data_clean.index, ticker_data_clean.values, 
             label='Stock Price', linewidth=2, color='black', alpha=0.7)
    
    # Plot buy signals
    buy_signals = [i for i, signal in enumerate(signals) if signal == 'buy']
    buy_prices = [ticker_data_clean.iloc[i] for i in buy_signals]
    buy_dates = [ticker_data_clean.index[i] for i in buy_signals]
    
    # Plot sell signals
    sell_signals = [i for i, signal in enumerate(signals) if signal == 'sell']
    sell_prices = [ticker_data_clean.iloc[i] for i in sell_signals]
    sell_dates = [ticker_data_clean.index[i] for i in sell_signals]
    
    if buy_dates:
        plt.scatter(buy_dates, buy_prices, color='green', s=100, marker='^', 
                   label='Buy Signal', zorder=5)
    
    if sell_dates:
        plt.scatter(sell_dates, sell_prices, color='red', s=100, marker='v', 
                   label='Sell Signal', zorder=5)
    
    # Shade periods when in position
    in_position = [i for i, pos in enumerate(positions) if pos == 'bought']
    if in_position:
        in_position_dates = [ticker_data_clean.index[i] for i in in_position]
        in_position_prices = [ticker_data_clean.iloc[i] for i in in_position]
        
        # Create a shaded region
        min_date = min(in_position_dates)
        max_date = max(in_position_dates)
        plt.axvspan(min_date, max_date, alpha=0.1, color='green', label='Holding Period')
    
    plt.title('Trading Signals and Positions', fontsize=16, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Price ($)', fontsize=12)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_performance_comparison(comparison, ticker_input):
    """
    Plot strategy performance vs buy-and-hold
    """
    plt.figure(figsize=(14, 8))
    
    # Plot both strategies
    plt.plot(comparison.index, comparison[ticker_input + ' Continuous Investing (%)'], 
             label='Buy-and-Hold Strategy', linewidth=2, color='blue', alpha=0.7)
    
    plt.plot(comparison.index, comparison[ticker_input + ' Indicative Investing (%)'], 
             label='Moving Average Strategy', linewidth=2, color='green', alpha=0.7)
    
    # Fill area where strategy outperforms
    plt.fill_between(comparison.index, 
                     comparison[ticker_input + ' Indicative Investing (%)'], 
                     comparison[ticker_input + ' Continuous Investing (%)'],
                     where=(comparison[ticker_input + ' Indicative Investing (%)'] > 
                            comparison[ticker_input + ' Continuous Investing (%)']),
                     color='green', alpha=0.2, label='Strategy Outperforming')
    
    plt.title('Strategy Performance Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Cumulative Return (%)', fontsize=12)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)
    
    # Add final return annotation
    final_strat = comparison[ticker_input + ' Indicative Investing (%)'].iloc[-1]
    final_bh = comparison[ticker_input + ' Continuous Investing (%)'].iloc[-1]
    
    plt.annotate(f'Strategy Final: {final_strat:.2f}%', 
                 xy=(0.02, 0.95), xycoords='axes fraction',
                 fontsize=12, bbox=dict(boxstyle="round,pad=0.3", facecolor="green", alpha=0.7))
    
    plt.annotate(f'Buy-and-Hold Final: {final_bh:.2f}%', 
                 xy=(0.02, 0.88), xycoords='axes fraction',
                 fontsize=12, bbox=dict(boxstyle="round,pad=0.3", facecolor="blue", alpha=0.7))
    
    plt.tight_layout()
    plt.show()

In [12]:
def collect_stock_data():
    """
    Collects stock data based on user inputs
    """
    print("=" * 60)
    print("STOCK TRADING STRATEGY ANALYZER")
    print("=" * 60)
    
    # Get user inputs
    ticker_input = input("Please enter the stock ticker you have chosen to invest in: \n").strip().upper()
    
    print("\n" + "-" * 60)
    start_date = input("Enter the start date (format: YYYY-MM-DD):\n").strip()
    
    print("\n" + "-" * 60)
    end_date = input("Enter the end date (format: YYYY-MM-DD):\n").strip()
    
    print("\n" + "-" * 60)
    print("MOVING AVERAGE PERIODS")
    print("-" * 60)
    ma_list = []
    
    # Get first moving average period
    while True:
        try:
            ma_value = int(input("What would you like the 1st period to be for your moving average?\n"))
            if ma_value > 0:
                ma_list.append(ma_value)
                break
            else:
                print("Please enter a positive integer.")
        except ValueError:
            print("Invalid input. Please enter a valid integer.")
    
    # Get second moving average period
    while True:
        try:
            ma_value = int(input("What would you like the 2nd period to be for your moving average?\n"))
            if ma_value > 0 and ma_value != ma_list[0]:
                ma_list.append(ma_value)
                break
            elif ma_value == ma_list[0]:
                print(f"Please enter a different value than the first period ({ma_list[0]}).")
            else:
                print("Please enter a positive integer.")
        except ValueError:
            print("Invalid input. Please enter a valid integer.")
    
    # Sort MA periods (shorter first)
    ma_list.sort()
    
    print(f"\nSelected moving average periods: {ma_list[0]}-day and {ma_list[1]}-day")
    
    # Download stock data
    print(f"\nDownloading data for {ticker_input}...")
    try:
        # Add 1 day to end_date to include the end date in the data
        import datetime
        end_date_adj = (pd.to_datetime(end_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
        
        ticker_data_full = yf.download(ticker_input, start=start_date, end=end_date_adj, progress=False)
        
        if ticker_data_full.empty:
            print("No data found. Please check your ticker symbol and date range.")
            return None, None, None
        
        # Check for available columns and use appropriate price column
        available_columns = ticker_data_full.columns.tolist()
        
        # Try to use Adjusted Close, fall back to Close if not available
        if 'Adj Close' in available_columns:
            ticker_data = ticker_data_full['Adj Close'].copy()
            price_type = "Adjusted Close"
        elif 'Close' in available_columns:
            ticker_data = ticker_data_full['Close'].copy()
            price_type = "Close"
        else:
            # If neither is available, use the first numeric column
            numeric_cols = ticker_data_full.select_dtypes(include=[np.number]).columns
            if len(numeric_cols) > 0:
                ticker_data = ticker_data_full[numeric_cols[0]].copy()
                price_type = numeric_cols[0]
            else:
                print("No numeric price data found in the downloaded data.")
                return None, None, None
        
        print(f"Successfully downloaded {len(ticker_data)} days of data")
        print(f"Date range: {ticker_data.index[0].date()} to {ticker_data.index[-1].date()}")
        print(f"Using price data from: {price_type}")
        
        # Show available columns for debugging
        print(f"\nAvailable data columns: {', '.join(available_columns)}")
        
        return ticker_input, ticker_data, ma_list
        
    except Exception as e:
        print(f"Error downloading data: {str(e)}")
        print("Common issues:")
        print("1. Invalid ticker symbol")
        print("2. Invalid date format (use YYYY-MM-DD)")
        print("3. Market was closed on the specified dates")
        print("4. Network connection issue")
        return None, None, None

In [14]:
# Execute the complete analysis
results = run_complete_analysis()

# Optionally save results to CSV
if results:
    print("\n" + "-" * 60)
    save_option = input("Would you like to save the results to CSV files? (yes/no): ")
    
    if save_option.lower() == 'yes':
        ticker = results['ticker']
        
        # Save price data with moving averages
        combined_data = pd.concat([results['data'], results['moving_averages']], axis=1)
        combined_data.to_csv(f'{ticker}_price_ma_data.csv')
        
        # Save trading signals
        signals_df = pd.DataFrame({
            'Date': results['data'].index,
            'Price': results['data'].values,
            'Signal': results['signals'],
            'Position': results['positions']
        })
        signals_df.set_index('Date', inplace=True)
        signals_df.to_csv(f'{ticker}_trading_signals.csv')
        
        # Save performance comparison
        results['comparison'].to_csv(f'{ticker}_performance_comparison.csv')
        
        # Save statistics
        results['statistics'].to_csv(f'{ticker}_statistics.csv')
        
        print(f"\nData saved to CSV files with prefix: {ticker}_")

MOVING AVERAGE TRADING STRATEGY ANALYSIS
STOCK TRADING STRATEGY ANALYZER


Please enter the stock ticker you have chosen to invest in: 
 MSFT



------------------------------------------------------------


Enter the start date (format: YYYY-MM-DD):
 2023-01-04



------------------------------------------------------------


Enter the end date (format: YYYY-MM-DD):
 2025-12-27



------------------------------------------------------------
MOVING AVERAGE PERIODS
------------------------------------------------------------


What would you like the 1st period to be for your moving average?
 5
What would you like the 2nd period to be for your moving average?
 3



Selected moving average periods: 3-day and 5-day

Successfully downloaded 748 days of data
Date range: 2023-01-04 to 2025-12-26
Using price data from: ('Close', 'MSFT')
Error downloading data: sequence item 0: expected str instance, tuple found
Common issues:
1. Invalid ticker symbol
2. Invalid date format (use YYYY-MM-DD)
3. Market was closed on the specified dates
4. Network connection issue
Exiting due to data collection error.
